### Import dependencies

In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
pip install mlxtend

     |████████████████████████████████| 1.4 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 305 kB/s eta 0:00:011
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-ml 1.2.0 requires numpy>=1.17.3, but you have numpy 1.17.2 which is incompatible.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from mlxtend.classifier import StackingCVClassifier
import warnings
warnings.filterwarnings('ignore')

### Read files

In [5]:
os.chdir("/Users/Tony/Desktop/IPBA - Business Analytics/Logistic Regression")
dat1= pd.read_csv('adult.csv')

In [6]:
dat=dat1[0:500]

In [7]:
dat.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


### Explore data

In [8]:
dat.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,37.338000,188303.492000,10.008000,1470.008000,70.916000,40.272000
std,13.652712,101198.741043,2.543398,9298.308254,354.580591,11.981366
min,17.000000,20308.000000,2.000000,0.000000,0.000000,5.000000
25%,26.000000,110718.250000,9.000000,0.000000,0.000000,38.750000
50%,35.000000,177832.000000,10.000000,0.000000,0.000000,40.000000
75%,46.000000,238286.500000,12.000000,0.000000,0.000000,45.000000
max,80.000000,599057.000000,16.000000,99999.000000,2415.000000,99.000000


In [9]:
dat.isnull().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [10]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 15 columns):
age                500 non-null int64
workclass          500 non-null object
fnlwgt             500 non-null int64
education          500 non-null object
educational-num    500 non-null int64
marital-status     500 non-null object
occupation         500 non-null object
relationship       500 non-null object
race               500 non-null object
gender             500 non-null object
capital-gain       500 non-null int64
capital-loss       500 non-null int64
hours-per-week     500 non-null int64
native-country     500 non-null object
income             500 non-null object
dtypes: int64(6), object(9)
memory usage: 58.7+ KB


In [11]:
col=dat.columns[dat.dtypes=="object"].tolist()
print(col)
type(col)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']


list

In [12]:
print(dat.workclass.unique())

['Private' 'Local-gov' '?' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc']


In [13]:
print(dat.education.unique())

['11th' 'HS-grad' 'Assoc-acdm' 'Some-college' '10th' 'Prof-school'
 '7th-8th' 'Bachelors' 'Masters' 'Doctorate' '5th-6th' 'Assoc-voc' '9th'
 '12th' '1st-4th']


In [14]:
print(dat["marital-status"].unique())

['Never-married' 'Married-civ-spouse' 'Widowed' 'Divorced' 'Separated'
 'Married-spouse-absent' 'Married-AF-spouse']


In [15]:
print(dat["occupation"].unique())

['Machine-op-inspct' 'Farming-fishing' 'Protective-serv' '?'
 'Other-service' 'Prof-specialty' 'Craft-repair' 'Adm-clerical'
 'Exec-managerial' 'Tech-support' 'Sales' 'Priv-house-serv'
 'Transport-moving' 'Handlers-cleaners' 'Armed-Forces']


In [16]:
print(dat["relationship"].unique())

['Own-child' 'Husband' 'Not-in-family' 'Unmarried' 'Wife' 'Other-relative']


In [17]:
print(dat["race"].unique())

['Black' 'White' 'Asian-Pac-Islander' 'Other' 'Amer-Indian-Eskimo']


In [18]:
print(dat["gender"].unique())

['Male' 'Female']


In [19]:
print(dat["native-country"].unique())

['United-States' '?' 'Peru' 'Guatemala' 'Mexico' 'Dominican-Republic'
 'Ireland' 'Germany' 'Philippines' 'Thailand' 'Haiti' 'El-Salvador'
 'Puerto-Rico' 'Vietnam' 'South' 'Columbia' 'Japan' 'India' 'Cambodia'
 'Poland' 'Laos']


#### Creating Dependent Variable

In [20]:
print(dat["income"].unique())

['<=50K' '>50K']


In [21]:
# Creating dependent variable list with categories
y = pd.DataFrame(np.where(dat["income"] == ">50K" , 1, 0))
#y=dat["income"].map(lambda x: 1 if x== ">50K" else 0)

In [22]:
y[0:5]

,0
0,0
1,0
2,1
3,1
4,0


In [23]:
len(y)

500

### Preparing Train and Test Data

In [24]:
X=dat.drop(['income'],axis=1)

In [25]:
X.head(2)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States


In [26]:
X.shape

(500, 14)

In [27]:
X=pd.get_dummies(X)

In [28]:
X.shape

(500, 84)

In [29]:
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Laos,native-country_Mexico,native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Puerto-Rico,native-country_South,native-country_Thailand,native-country_United-States,native-country_Vietnam
0,25,226802,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,38,89814,9,0,0,50,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,44,160323,10,7688,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,18,103497,10,0,0,30,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [30]:
y.shape

(500, 1)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=200)

### Model fitting using Support Vector Classification.

In [32]:
from sklearn.svm import SVC
model = SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.792

In [33]:
rssvc = RandomizedSearchCV(SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rssvc.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='auto', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=2, n_jobs=None,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [34]:
pd.DataFrame(rssvc.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.808
1,20,linear,0.808


In [35]:
column_names = ["Model", "params", "mean_test_score"]
scoretab = pd.DataFrame(columns = column_names)
scoretab

,Model,params,mean_test_score


In [36]:
stsvc= pd.DataFrame(rssvc.cv_results_)[['params','mean_test_score']]
stsvc["Model"]=stsvc["params"].map (lambda x:"Support Vector")
stsvc

,params,mean_test_score,Model
0,"{'kernel': 'linear', 'C': 10}",0.808,Support Vector
1,"{'kernel': 'linear', 'C': 20}",0.808,Support Vector


In [37]:
rssvc.cv_results_

{'mean_fit_time': array([24.11034317, 14.38270159]),
 'std_fit_time': array([14.07451144,  4.72007134]),
 'mean_score_time': array([0.00105596, 0.00095344]),
 'std_score_time': array([5.95040426e-05, 1.00118204e-04]),
 'param_kernel': masked_array(data=['linear', 'linear'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[10, 20],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'linear', 'C': 10}, {'kernel': 'linear', 'C': 20}],
 'split0_test_score': array([0.8, 0.8]),
 'split1_test_score': array([0.8 , 0.82]),
 'split2_test_score': array([0.88, 0.88]),
 'split3_test_score': array([0.78, 0.78]),
 'split4_test_score': array([0.78, 0.76]),
 'mean_test_score': array([0.808, 0.808]),
 'std_test_score': array([0.03709447, 0.04118252]),
 'rank_test_score': array([1, 1], dtype=int32)}

### Model fitting using Random Forest Classifier

In [38]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=100, random_state=200)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.84

In [39]:
rsrfc = RandomizedSearchCV(RandomForestClassifier(), {
        'n_estimators': [50,100,250,500],
        'max_depth': [2,3,4,5]
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rsrfc.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [40]:
strfc= pd.DataFrame(rsrfc.cv_results_)[['params','mean_test_score']]
strfc["Model"]=strfc["params"].map (lambda x:"Random Forest")
strfc

,params,mean_test_score,Model
0,"{'n_estimators': 500, 'max_depth': 4}",0.844,Random Forest
1,"{'n_estimators': 100, 'max_depth': 5}",0.868,Random Forest


In [41]:
rsrfc.cv_results_

{'mean_fit_time': array([0.26521807, 0.05472274]),
 'std_fit_time': array([0.00159872, 0.00024313]),
 'mean_score_time': array([0.01228075, 0.00288758]),
 'std_score_time': array([1.52665759e-04, 2.49357112e-05]),
 'param_n_estimators': masked_array(data=[500, 100],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[4, 5],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 500, 'max_depth': 4},
  {'n_estimators': 100, 'max_depth': 5}],
 'split0_test_score': array([0.84, 0.86]),
 'split1_test_score': array([0.86, 0.9 ]),
 'split2_test_score': array([0.84, 0.86]),
 'split3_test_score': array([0.8 , 0.84]),
 'split4_test_score': array([0.88, 0.88]),
 'mean_test_score': array([0.844, 0.868]),
 'std_test_score': array([0.026533  , 0.02039608]),
 'rank_test_score': array([2, 1], dtype=int32)}

### Model fitting using Decision Tree Classifier

In [42]:
rsdtc = RandomizedSearchCV(DecisionTreeClassifier(), {
        'max_features': [None, "auto", "sqrt", "log2"],
        'max_depth': [2,3,4,5]
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rsdtc.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
          

In [43]:
stdtc=pd.DataFrame(rsdtc.cv_results_)[['params','mean_test_score']]
stdtc["Model"]=stdtc["params"].map (lambda x:"Decision Tree")
stdtc

,params,mean_test_score,Model
0,"{'max_features': 'log2', 'max_depth': 4}",0.812,Decision Tree
1,"{'max_features': 'sqrt', 'max_depth': 5}",0.816,Decision Tree


### Model fitting using KNeighbors Classifier

In [44]:
rsknc = RandomizedSearchCV(KNeighborsClassifier(), {
        'n_neighbors': [6,7,8,9,10,11,12,14,16,18,20],
        'algorithm': ['auto'], 
        'weights': ['uniform', 'distance'],
        'leaf_size': list(range(1,50,5))
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rsknc.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='deprecated', n_iter=2, n_jobs=None,
                   param_distributions={'algorithm': ['auto'],
                                        'leaf_size': [1, 6, 11, 16, 21, 26, 31,
                                                      36, 41, 46],
                                        'n_neighbors': [6, 7, 8, 9, 10, 11, 12,
                                                        14, 16, 18, 20],
                                        'weights': ['uniform', 'distance']},
                   pre_dispatch='

In [45]:
stknc=pd.DataFrame(rsknc.cv_results_)[['params','mean_test_score']]
stknc["Model"]=stknc["params"].map (lambda x:"K Neighbour")
stknc

,params,mean_test_score,Model
0,"{'weights': 'uniform', 'n_neighbors': 20, 'lea...",0.792,K Neighbour
1,"{'weights': 'distance', 'n_neighbors': 20, 'le...",0.760,K Neighbour


### Model fitting using Logistic Regression

In [46]:
rslgr = RandomizedSearchCV(LogisticRegression(), {
        'C': [0.1, 1, 10, 100]
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rslgr.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=2, n_jobs=None,
                   param_distributions={'C': [0.1, 1, 10, 100]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [47]:
stlgr=pd.DataFrame(rslgr.cv_results_)[['params','mean_test_score']]
stlgr["Model"]=stlgr["params"].map (lambda x:"Logistic Regression")
stlgr

,params,mean_test_score,Model
0,{'C': 1},0.820,Logistic Regression
1,{'C': 0.1},0.824,Logistic Regression


### Model fitting using Gradient Boosting Classifier

In [48]:
rsgbc = RandomizedSearchCV(GradientBoostingClassifier(), {
        'learning_rate': [0.1, 0.2, 0.5],
        'n_estimators': [50, 100, 150],
        'max_depth': [2,3]
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rsgbc.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                        criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                    

In [49]:
stgbc=pd.DataFrame(rsgbc.cv_results_)[['params','mean_test_score']]
stgbc["Model"]=stgbc["params"].map (lambda x:"Gradient Boosting")
stgbc

,params,mean_test_score,Model
0,"{'n_estimators': 100, 'max_depth': 3, 'learnin...",0.856,Gradient Boosting
1,"{'n_estimators': 150, 'max_depth': 2, 'learnin...",0.868,Gradient Boosting


### Model fitting using XGB Classifier

In [50]:
rsxgb = RandomizedSearchCV(XGBClassifier(), {
        'learning_rate': [0.1, 0.2, 0.3],
        'n_estimators': [100, 150, 500, 1000],
        'max_depth': [3,4,5],
        'gamma': [0, 0.2, 0.5]
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rsxgb.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, scale_pos_weight=1,
                                           seed=None, silent=None, subsample=1,
                                           verbosity=1),
                   iid='de

In [51]:
stxgb=pd.DataFrame(rsxgb.cv_results_)[['params','mean_test_score']]
stxgb["Model"]=stxgb["params"].map (lambda x:"XGB")
stxgb

,params,mean_test_score,Model
0,"{'n_estimators': 100, 'max_depth': 3, 'learnin...",0.848,XGB
1,"{'n_estimators': 150, 'max_depth': 5, 'learnin...",0.844,XGB


### Model fitting using Gaussian Naive Bayes 

In [52]:
rsgnb = RandomizedSearchCV(GaussianNB(),{"priors":[None], "var_smoothing":[1e-09]}, cv=5, 
    return_train_score=False)
rsgnb.fit(X_test, y_test)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'priors': [None],
                                        'var_smoothing': [1e-09]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [53]:
stgnb=pd.DataFrame(rsgnb.cv_results_)[['params','mean_test_score']]
stgnb["Model"]=stgnb["params"].map (lambda x:"Gaussian Naive Bayes")
stgnb

,params,mean_test_score,Model
0,"{'var_smoothing': 1e-09, 'priors': None}",0.82,Gaussian Naive Bayes


### Tabulating summary of mean scores

In [54]:
scoretab=scoretab.append(stsvc) 
scoretab=scoretab.append(strfc) 
scoretab=scoretab.append(stdtc) 
scoretab=scoretab.append(stknc) 
scoretab=scoretab.append(stlgr) 
scoretab=scoretab.append(stgbc) 
scoretab=scoretab.append(stxgb)
scoretab=scoretab.append(stgnb)
scoretab.sort_values('mean_test_score', ascending=False)

,Model,mean_test_score,params
1,Random Forest,0.868,"{'n_estimators': 100, 'max_depth': 5}"
1,Gradient Boosting,0.868,"{'n_estimators': 150, 'max_depth': 2, 'learnin..."
0,Gradient Boosting,0.856,"{'n_estimators': 100, 'max_depth': 3, 'learnin..."
0,XGB,0.848,"{'n_estimators': 100, 'max_depth': 3, 'learnin..."
0,Random Forest,0.844,"{'n_estimators': 500, 'max_depth': 4}"
1,XGB,0.844,"{'n_estimators': 150, 'max_depth': 5, 'learnin..."
1,Logistic Regression,0.824,{'C': 0.1}
0,Logistic Regression,0.820,{'C': 1}
0,Gaussian Naive Bayes,0.820,"{'var_smoothing': 1e-09, 'priors': None}"
1,Decision Tree,0.816,"{'max_features': 'sqrt', 'max_depth': 5}"


### Building a pipeline for Parameter Tuning

In [55]:
# Initialising the pipeline with one estimator  
pipe = Pipeline(steps=[('classifier', SVC())])

In [62]:
# Add a dict of classifier and related parameters in list

params_grid = [{
                'classifier':[SVC()],
                'classifier__C': [0.1, 1, 10, 100],
                'classifier__gamma': [0.001, 0.0001]
                },
              {
                'classifier': [DecisionTreeClassifier()],
                'classifier__max_depth': [2,3,4],
                'classifier__max_features': [None, "sqrt", "log2"]
              },
              {
                'classifier': [KNeighborsClassifier()],
                'classifier__n_neighbors': [6,7,8,9,10,11,12,14,16,18,20],
                'classifier__algorithm': ['auto'],
                'classifier__weights': ['uniform', 'distance'],
                'classifier__leaf_size': list(range(1,50,5))
              }, 
              {
                'classifier': [LogisticRegression()],
                'classifier__C': [0.1, 1, 10, 100],
              },
              {
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [50, 100, 150],
                'classifier__max_depth': [2, 3, 4, 5],
              },
              {
                'classifier': [GradientBoostingClassifier()],
                'classifier__learning_rate': [0.1, 0.2, 0.5],
                'classifier__n_estimators': [50, 100, 150],
                'classifier__max_depth': [2,3]
              },
              {
                'classifier': [XGBClassifier()],
                'classifier__learning_rate': [0.1, 0.2, 0.3],
                'classifier__n_estimators': [100, 150, 500, 1000],
                'classifier__max_depth': [3,4,5],
                'classifier__gamma': [0, 0.2, 0.5]  
              },
              {
                'classifier': [GaussianNB()]
              }]


In [75]:
gd = GridSearchCV(estimator=pipe, param_grid=params_grid, cv=5, scoring='accuracy')
gd.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('classifier',
                                        SVC(C=1.0, break_ties=False,
                                            cache_size=200, class_weight=None,
                                            coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='scale',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='deprecated', n_jobs...
                                                       seed=None, silent=None,
                                                       subsample

In [78]:
# Extract the best estimator and refit if needed
best_model = gd.best_estimator_

# Optionally re-fit the best model on new training data
best_model.fit(X_train, y_train)  # If needed, although it's already fitted by GridSearchCV

Pipeline(memory=None,
         steps=[('classifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=4, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=50, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [80]:
y_pred_1 = gd.best_estimator_.predict(X_test)

In [81]:
best_model = KNeighborsClassifier(algorithm='auto', 
                                  leaf_size=26,
                                  metric='minkowski', 
                                  metric_params=None,
                                  n_neighbors=18, 
                                  p=2,
                                  weights='uniform')



In [82]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)